# Detección y caracterización de caras

### Entrenamiento para detección de attributos

In [11]:
import torch
import torch.nn as nn
from pygments.styles.dracula import background
from sympy import horner
from sympy.physics.vector import get_motion_params
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

EPOCHS = 35
DATASET_BASE_DIRECTORY = "../emotion-recognition-dataset/"
BATCH_SIZE = 32
NUM_CLASSES = 8
last_accuracy = 0.5844

# --- Data transforms ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Load your dataset
train_dataset = datasets.ImageFolder(DATASET_BASE_DIRECTORY + "train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = datasets.ImageFolder(DATASET_BASE_DIRECTORY + "val", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# --- Load EfficientNet ---
weights = models.EfficientNet_B2_Weights.IMAGENET1K_V1
emotion_model = models.efficientnet_b2(weights=weights)
# Replace classifier for 8 emotion classes
num_features = emotion_model.classifier[1].in_features
emotion_model.classifier[1] = nn.Linear(num_features, NUM_CLASSES)


for param in emotion_model.features[:5].parameters():  # adjust slice as needed
    param.requires_grad = False

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotion_model = emotion_model.to(device)

# --- Loss & optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(emotion_model.parameters(), lr=1e-4)

# --- Training loop (simplified) ---
for epoch in range(EPOCHS):
    emotion_model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = emotion_model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} done")


# --- Validation loop ---
emotion_model.eval()  # set model to evaluation mode
running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # disable gradient computation
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = emotion_model(imgs)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * imgs.size(0)

        # Compute number of correct predictions
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

avg_loss = running_loss / total
accuracy = correct / total

print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy*100:.2f}%")

if accuracy > last_accuracy:
    torch.save(emotion_model.state_dict(), "emotion-efficientnet-weights.pth")


Epoch 0 done
Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done
Epoch 6 done
Epoch 7 done
Epoch 8 done
Epoch 9 done
Epoch 10 done
Epoch 11 done
Epoch 12 done
Epoch 13 done
Epoch 14 done
Epoch 15 done
Epoch 16 done
Epoch 17 done
Epoch 18 done
Epoch 19 done
Epoch 20 done
Epoch 21 done
Epoch 22 done
Epoch 23 done
Epoch 24 done
Epoch 25 done
Epoch 26 done
Epoch 27 done
Epoch 28 done
Epoch 29 done
Epoch 30 done
Epoch 31 done
Epoch 32 done
Epoch 33 done
Epoch 34 done
Validation Loss: 1.9946, Accuracy: 60.50%


### Filtro de emociones

In [1]:
import cv2
import filters
from deepface import DeepFace

# happy, angry, fear, neutral, sad, disgust, surprise
filters_dict = {
    "happy": filters.HappyFilter("resources/images/aureola.png"),
    "angry": filters.AngryFilter("resources/images/devil-horns.png"),
    "sad": filters.SadFilter("resources/images/tears.png"),
    "fear": filters.FearFilter("resources/images/cold-sweat.png"),
    "surprise": filters.SurpriseFilter("resources/images/surprise.png", "resources/images/surprised_eye.png"),
}

def predict_emotion(img, face_data):
    x, y, w, h = face_data['facial_area']['x'], face_data['facial_area']['y'], face_data['facial_area']['w'], face_data['facial_area']['h']
    cropped_face_rgb = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2RGB)
    objs = DeepFace.analyze(cropped_face_rgb, actions = ['emotion'], enforce_detection=False)
    dominant_emotion = objs[0]['dominant_emotion']
    cv2.putText(img, f"Emotion: {dominant_emotion}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return dominant_emotion


cap = cv2.VideoCapture(0)
ret, frame = cap.read()
key = 0

while ret and key != 27:
    try:
        for face in DeepFace.extract_faces(frame, detector_backend="yolov8", enforce_detection=False):
            filters_dict.get(predict_emotion(frame, face), filters.NoFilter()).apply(frame, face)

    except Exception as e:
        print(f"error: {e}")

    cv2.imshow("Video", frame)
    ret, frame = cap.read()
    key = cv2.waitKey(1) & 0xFF

# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()

2025-11-16 15:13:36.004541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 15:13:36.046603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 15:13:37.038042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 15:13:41.582490: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocat

### Filtro libre

##### Función para guardar un gif en memoria

In [4]:
import cv2
import filters
import mediapipe as mp
import time
import imageio

def read_gif(path, width, height):
    gif = imageio.mimread(path)
    frames = [cv2.resize(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), (width, height)) for frame in gif]
    return frames

In [16]:
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True)

cap = cv2.VideoCapture("resources/test/face_1.mp4")
ret, frame = cap.read()
key = 0
galaxy_filter = filters.GalaxyFilter()
debug_filter = filters.MediapipeDebugFilter()

frame_number = 0
w,h,_ = frame.shape

FRAME_WIDTH = 520
FRAME_HEIGHT = int(FRAME_WIDTH * h/w)

background_gif_frames = read_gif("resources/images/matrix_background.gif", FRAME_WIDTH, FRAME_HEIGHT)
while ret and key != 27:
    try:
        # Just for this video
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = background_gif_frames[frame_number].copy()

        for face in mp_face_mesh.process(frame_rgb).multi_face_landmarks:
            debug_filter.apply(frame, face)

        # Just for this video
        time.sleep(0.02)

    except Exception as e:
        print(f"error: {e}")

    cv2.imshow("Video", frame)
    ret, frame = cap.read()
    key = cv2.waitKey(1) & 0xFF
    frame_number = (frame_number + 1) % len(background_gif_frames)


# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1763396264.978581   19180 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763396264.979922   20025 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.9), renderer: Mesa Intel(R) UHD Graphics (TGL GT1)
W0000 00:00:1763396264.982496   20021 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763396264.988454   20020 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
